### AI Lab -  Assignment 6 - CH22M503

#### Author - Aloy Banerjee
###### Roll No. CH22M503

## Question:
###### Use the dataset provided for classification in assignment IV.
###### (a) Build and train Support Vector Machine (SVM) classifiers for the given task of classification.Train two models, one with the ‘polynomial’ kernel and the other one with ‘rbf’ kernel. For the polynomial kernel, the choice of degree has to come from hyper-parameter optimization. Compare the model’s performance on the basis of accuracy and ROC-AUC metrics
###### (b) Train a neural network model for the same task of classification. Report its performance
###### (c) (Optional) Train the above-mentioned neural network model with comparatively less overfitting.(Hint: Use drop-out layers (or) early call back)

#### Reference 

In [1]:
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
#https://stackoverflow.com/questions/59666138/sklearn-roc-auc-score-with-multi-class-ovr-should-have-none-average-available
#https://stackoverflow.com/questions/39685740/calculate-sklearn-roc-auc-score-for-multi-class

#### Importing Library

In [ ]:
#Import library 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import math
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.metrics import classification_report,confusion_matrix,roc_curve,auc,RocCurveDisplay,accuracy_score,make_scorer,roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
sns.set(context="notebook", palette="viridis", style="darkgrid", font_scale=1.0, color_codes=True,rc={'figure.figsize':(14,10)})

#### Common variable declaration

In [ ]:
workingDir = 'E:\\Document\\M-Tech\\M-Tech Class\\2nd Trimester\\Assignment\\AI Lab\\Assignment 6'
dataset_fileName = 'ch22m50 (3).csv'
modelObj = []
seed = 42
testorvalidation_size = 0.2
degree = np.arange(0,10,1)
C_value = [0.001, 0.01, 0.1, 1, 10]
gamma_range = ['scale', 'auto']
cross_validation = 10
maximum_iteration = 500
epochs = 1000
round_precesion = 2
param_grid = {
    'C': C_value,
    'degree': degree,
    'gamma': gamma_range,
    'coef0': [0.0, 1.0, 2.0, 3.0]
}

#### Common function

In [ ]:
def data_load(input_x, output_y):
    X_train_val, X_test, y_train_val, y_test = train_test_split(input_x, output_y, test_size=testorvalidation_size, random_state=seed)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=testorvalidation_size, random_state=seed)
    return X_train, X_val, X_test, y_train, y_val, y_test
def one_hot_encode_labels(Y):
    return np.eye(4)[Y]
def flatten_array(Y):
    return Y.reshape((len(Y), np.prod(Y.shape[1:])))
def get_classification_metrics(actual_labels, predicted_labels):
    unique_labels = np.sort(np.unique(actual_labels))
    actual_labels_np = actual_labels.numpy()
    predicted_labels_np = predicted_labels.numpy()
    confusion_matrix = np.zeros((len(unique_labels), len(unique_labels)))
    classification_report = np.zeros((len(unique_labels), 5))    
    # Calculate confusion matrix
    for i in range(len(unique_labels)):
        for j in range(len(unique_labels)):
            confusion_matrix[i, j] = np.sum((actual_labels_np == unique_labels[i]) & (predicted_labels_np == unique_labels[j]))            
    # Calculate category accuracy
    category_accuracy = confusion_matrix.diagonal() / confusion_matrix.sum(axis=1)    
    weighted_acc = 0
    # Calculate metrics for each category
    for j in range(len(unique_labels)):        
        actual_label, predicted_label = (actual_labels_np == unique_labels[j]), (predicted_labels_np == unique_labels[j])
        true_positives, false_positives, false_negatives = np.sum(actual_label & predicted_label), np.sum(~actual_label & predicted_label), np.sum(actual_label & ~predicted_label)
        count = np.sum(actual_labels_np == unique_labels[j])
        precision = true_positives / (true_positives + false_positives)
        recall = true_positives / (true_positives + false_negatives)
        f1_score = 2 * (precision * recall) / (precision + recall) 
        classification_report[j, :] = np.round([precision, recall, f1_score, category_accuracy[j], count], round_precesion)
        weighted_acc += (f1_score * count)        
    weighted_acc = weighted_acc / len(actual_labels)    
    return weighted_acc, classification_report

##### Loading Dataset

In [214]:
#Change the working directory 
os.chdir(workingDir)
#Load the data
dataset = pd.read_csv(dataset_fileName)

##### Display info of dataset

In [215]:
display(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512 entries, 0 to 511
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x_0     512 non-null    float64
 1   x_1     512 non-null    float64
 2   x_2     512 non-null    float64
 3   x_3     512 non-null    float64
 4   x_4     512 non-null    float64
 5   x_5     512 non-null    float64
 6   x_6     512 non-null    float64
 7   x_7     512 non-null    float64
 8   x_8     512 non-null    float64
 9   x_9     512 non-null    float64
 10  x_10    512 non-null    float64
 11  x_11    512 non-null    float64
 12  x_12    512 non-null    float64
 13  x_13    512 non-null    float64
 14  x_14    512 non-null    float64
 15  x_15    512 non-null    float64
 16  x_16    512 non-null    float64
 17  x_17    512 non-null    float64
 18  x_18    512 non-null    float64
 19  x_19    512 non-null    float64
 20  x_20    512 non-null    float64
 21  x_21    512 non-null    float64
 22  x_

None

##### Display statistical information of each column of dataset 

In [216]:
display(dataset.describe())

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_20,x_21,x_22,x_23,x_24,x_25,x_26,x_27,x_28,y
count,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,...,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000
mean,2.439131,37.565795,-2.129354,-42.896385,-19.541749,0.774351,-3.318514,1.234039,-37.084747,44.457026,...,3.954715,7.062185,-1.984638,-0.041593,45.802623,-6.156434,20.336928,7.686032,46.116476,1.396484
std,79.668269,149.000493,70.127878,270.463154,169.129888,5.008753,82.998787,32.479082,165.128140,147.647250,...,136.591423,94.580884,35.141745,8.054942,210.354097,44.965474,185.172480,33.899699,222.968262,0.869705
min,-218.014426,-420.663489,-200.408297,-805.895235,-487.537653,-13.031117,-211.774697,-104.569640,-567.697872,-370.866192,...,-387.207794,-251.840570,-146.937805,-22.463712,-568.799810,-129.243517,-593.246183,-101.722870,-545.635486,0.000000
25%,-49.223112,-61.024707,-46.175353,-209.804499,-132.511157,-2.415076,-64.127241,-20.197596,-149.740462,-53.430691,...,-87.250875,-56.243393,-25.552702,-5.949824,-89.267524,-36.253059,-99.238991,-17.600614,-116.271767,1.000000
50%,12.155885,41.328052,-2.193064,-39.650751,-15.800571,0.672129,-9.072157,0.849871,-35.757947,41.890772,...,10.971585,3.737968,-3.289748,0.288603,55.123325,-7.736827,17.342673,8.372459,30.868008,1.000000
75%,60.213974,141.277355,46.198129,141.936365,96.539653,4.211238,50.150721,23.711935,75.528969,143.915499,...,87.445776,69.919878,21.889408,5.714857,190.281398,23.727257,144.533145,30.951389,204.465337,2.000000
max,199.703493,484.171709,211.004193,720.820175,441.163606,16.231479,253.060961,122.069373,478.614180,552.311766,...,461.229792,320.096620,106.122619,22.005079,675.198210,142.665835,613.995137,139.990511,766.813592,3.000000


##### Display the given dataset 

In [217]:
display(dataset)

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_20,x_21,x_22,x_23,x_24,x_25,x_26,x_27,x_28,y
0,16.920655,-42.645832,120.989582,204.158654,146.482712,10.931416,-148.843296,27.660603,-84.502239,12.294844,...,-62.325705,38.713472,-35.038464,-9.714632,36.987391,8.891164,-434.626733,42.773671,217.002701,3
1,33.112697,141.934286,-42.908480,272.840275,3.240221,2.900615,83.384757,38.419407,5.996595,-74.794468,...,-63.303622,69.658188,50.411234,-14.496756,62.013898,23.602845,-56.753575,55.810014,132.221649,1
2,-0.719035,162.356669,-25.289948,-38.063813,-45.747704,-1.768817,-34.906378,19.047969,-122.525011,37.495460,...,19.455654,99.033783,-14.787519,-2.837963,80.574733,37.568538,-55.843178,24.805701,-66.399042,1
3,41.114839,32.865930,3.375600,248.982501,145.736013,0.144523,-100.467590,-14.692076,-224.750326,5.088222,...,28.051349,-69.535135,6.634928,-0.086851,243.915623,-42.572195,349.780139,22.955581,115.350261,2
4,-98.954896,100.334688,22.108893,-509.285564,77.418419,-2.567594,27.837708,41.203306,195.284077,-47.101180,...,103.337332,100.588271,-40.951918,3.262511,174.265657,38.307570,-363.908029,52.254404,362.320497,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,-198.199135,-78.961783,101.635923,87.151381,85.924719,2.857203,-114.279865,122.069373,134.942756,36.458869,...,173.351132,-66.298207,-48.326594,-11.743241,158.089972,-41.033300,24.813333,87.438227,766.813592,2
508,109.290428,191.480509,12.303618,-305.932511,-40.519626,-3.313111,-69.436370,21.865336,-53.042458,69.572077,...,39.230760,-63.706061,-37.408941,7.334285,6.379320,-39.800946,-313.897782,-21.168608,-248.854618,3
509,-19.049404,9.356148,34.453434,100.074962,-239.541858,-0.075924,-28.929118,-9.993400,111.402361,-159.851843,...,106.780895,-174.278694,24.498491,0.962552,16.108659,-92.369189,-7.323660,-51.172846,412.148554,1
510,33.054867,-122.689085,-23.861270,417.833334,-26.519607,-3.545035,-97.259415,-5.704583,295.703536,156.880560,...,-239.699164,102.871363,-3.690261,14.396206,-37.507866,39.392993,-277.306626,-27.038689,488.231509,2


##### Checking for null or na values in data set 

In [218]:
dataset.isnull().isna().values.any()

False

In [219]:
input_x = dataset.drop('y', axis=1)
output_y = dataset[['y']] 

#### Check the unique target class

In [220]:
np.unique(output_y)

array([0, 1, 2, 3], dtype=int64)

#### 4 unique class is present in the dataset 

#### Q1 Build and train Support Vector Machine (SVM) classifiers for the given task of classification.Train two models, one with the ‘polynomial’ kernel and the other one with ‘rbf’ kernel. For the polynomial kernel, the choice of degree has to come from hyper-parameter optimization. Compare the model’s performance on the basis of accuracy and ROC-AUC metrics.

##### Splitting the dataset in train, validation and test set 

In [221]:
X_train, X_val, X_test, y_train, y_val, y_test = data_load(input_x, output_y)

#### Checking the dimension of the target output

In [222]:
print(f'Y_train Shape : {y_train.shape}')
print(f'Y_val Shape : {y_val.shape}')
print(f'Y_test Shape : {y_test.shape}')

Y_train Shape : (327, 1)
Y_val Shape : (82, 1)
Y_test Shape : (103, 1)


##### Define polynomial and rbf kernel SVC

In [223]:
#define the SVM classifiers with the polynomial and rbf kernels:
poly_svc_init = SVC(kernel='poly',probability=True)
rbf_svc_init = SVC(kernel='rbf',probability=True)

##### Hyper parameter optimization for evaluating best degree of polynomial kernel

In [ ]:
degree_range_param = {'degree': degree}
poly_svc__init_grid = GridSearchCV(poly_svc_init, param_grid, cv=cross_validation)
poly_svc__init_grid.fit(X_train, np.ravel(y_train))
poly_svc_best_param = poly_svc__init_grid.best_estimator_
print(f'Best hyperparameters for polynomial kernel SVM is: {poly_svc__init_grid.best_params_}')

##### Train SVM classifier using the best polynomial degree for polynomial kernel

In [199]:
poly_svc__init_grid.fit(X_train, np.ravel(y_train))

GridSearchCV(cv=10, estimator=SVC(kernel='poly', probability=True),
             param_grid={'degree': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])})

##### Train SVM classifier with RBF  kernel

In [206]:
rbf_svc_init.fit(X_train, np.ravel(y_train))

SVC(probability=True)

##### Prediction using Kernel SVM with polynomial kernel 

In [207]:
poly_y_pred = poly_svc__init_grid.predict(X_test)
print('Accuracy of polynomial kernel SVM:', accuracy_score(y_test, poly_y_pred))
print('ROC-AUC score of polynomial kernel SVM:', roc_auc_score(y_test, poly_svc__init_grid.predict_proba(X_test), multi_class='ovr'))

Accuracy of polynomial kernel SVM: 0.49514563106796117
ROC-AUC score of polynomial kernel SVM: 0.5575176230398127


##### Prediction using Kernel SVM with rbf kernel 

In [208]:
rbf_y_pred = rbf_svc_init.predict(X_test)
print('Accuracy of rbf kernel SVM:', accuracy_score(y_test, rbf_y_pred))
print('ROC-AUC score of rbf kernel SVM:', roc_auc_score(y_test, rbf_svc_init.predict_proba(X_test),multi_class='ovr'))

Accuracy of rbf kernel SVM: 0.47572815533980584
ROC-AUC score of rbf kernel SVM: 0.6055539879887744


## Conclusion : 
#### 1. Looking at the accuracy we can say that both kernel is providing accuracy in same range but looking at the given dataset  we confirm polynomial kernel is slightly better performing over RBF kernel.
#### 2. But while we are checking at the ROC-AUC score we can see that the score is higher in RBF kernel compare to polynomial kernel, it means that in case of our multi-class classification problem like given dataset, polynomial kernel model is performing nicely in classifying the different target classes but performing poorly to correctly classify instances within each class.

### As a conclusion we can confirm that the RBF kernel model is better at differentiating between the classes compare to polynomial kernel.

#### (b) Train a neural network model for the same task of classification. Report its performance.

In [85]:
activation ='relu'
solver = 'adam'

In [86]:
mlp_neural_network = MLPClassifier(hidden_layer_sizes=(128,), activation=activation, solver=solver, max_iter=maximum_iteration, random_state=seed)

In [87]:
mlp_neural_network.fit(X_train, np.ravel(y_train))

MLPClassifier(hidden_layer_sizes=(128,), max_iter=500, random_state=42)

In [88]:
y_train_prediction = mlp_neural_network.predict(X_train)
y_validation_prediction = mlp_neural_network.predict(X_val)
y_test_prediction = mlp_neural_network.predict(X_test)

print('Accuracy of neural network model : Train Dataset:', accuracy_score(y_train, y_train_prediction))
print('Accuracy of neural network model : Validation Dataset:', accuracy_score(y_val, y_validation_prediction))
print('Accuracy of neural network model: Test Dataset', accuracy_score(y_test, y_test_prediction))
#print('ROC-AUC score of neural network model:', roc_auc_score(y_test, mlp.predict_proba(X_test)[:, 1]),multi_class='ovr')


Accuracy of neural network model : Train Dataset: 1.0
Accuracy of neural network model : Validation Dataset: 0.4634146341463415
Accuracy of neural network model: Test Dataset 0.36893203883495146


## Conclusion  :
#### Looking at the accuracy it is clear that train accuracy is very high compared to validation or test accuracy, indicating there is over-fitting happen with the MLP classifier. We have several technique to overcome the over-fitting which we will try in the next section of the problem statement.

In [89]:
print('Train')
print(classification_report(y_train, y_train_prediction))
print('Validation')
print(classification_report(y_val, y_validation_prediction))
print('\n\nTest')
print(classification_report(y_test, y_test_prediction))

Train
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        42
           1       1.00      1.00      1.00       162
           2       1.00      1.00      1.00        83
           3       1.00      1.00      1.00        40

    accuracy                           1.00       327
   macro avg       1.00      1.00      1.00       327
weighted avg       1.00      1.00      1.00       327

Validation
              precision    recall  f1-score   support

           0       0.08      0.20      0.11         5
           1       0.62      0.65      0.63        40
           2       0.45      0.25      0.32        20
           3       0.38      0.35      0.36        17

    accuracy                           0.46        82
   macro avg       0.38      0.36      0.36        82
weighted avg       0.50      0.46      0.47        82



Test
              precision    recall  f1-score   support

           0       0.18      0.24      0.21        

In [90]:
X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
X_val_tensor = torch.tensor(X_val.to_numpy(), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32)

y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.long)
y_val_tensor = torch.tensor(y_val.to_numpy(), dtype=torch.long)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.long)

#### One hot encoding of target output

In [98]:
y_train_tensor = one_hot_encode_labels(y_train_tensor)
y_val_tensor = one_hot_encode_labels(y_val_tensor)
y_test_tensor = one_hot_encode_labels(y_test_tensor)

#### Flattening of target output

In [99]:
y_train_tensor = flatten_array(y_train_tensor)
y_val_tensor = flatten_array(y_val_tensor)
y_test_tensor = flatten_array(y_test_tensor)

#### Checking the dimension of tensor

In [104]:
print(f'Y_train tensor Shape : {y_train_tensor.shape}')
print(f'Y_val tensor Shape : {y_val_tensor.shape}')
print(f'Y_test tensor Shape : {y_test_tensor.shape}')

Y_train tensor Shape : (327, 4)
Y_val tensor Shape : (82, 4)
Y_test tensor Shape : (103, 4)


In [105]:
class ANNModel(nn.Module):
    def __init__(self):
        super(ANNModel, self).__init__()
        self.linear1 = nn.Linear(29, 128)        
        self.linear2 = nn.Linear(128, 128)
        self.linear3 = nn.Linear(128, 128)
        self.linear4 = nn.Linear(128, 4)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.relu(out)
        out = self.linear3(out)
        out = self.relu(out)
        out = self.linear4(out)
        return out   

In [106]:
class Trainer:
    
    def __init__(self, model, train_data, val_data, test_data):
        self.model = model
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data
        self.optimizer = optim.Adam(self.model.parameters(), lr=1e-2)
        self.criterion = nn.CrossEntropyLoss()

    def train(self, epochs):        
        for epoch in range(epochs): 
            running_loss = 0.0  
            data = self.train_data            
            inputs, labels = torch.tensor(data[0]).clone().detach(),torch.tensor(data[1]).clone().detach()
            self.optimizer.zero_grad()
            outputs = self.model(inputs)           
            loss = self.criterion(outputs, labels)
            loss.backward()
            self.optimizer.step()
            running_loss += loss.item()
            if epoch % 50 == 49:
                print(f"Epoch {epoch+1}, Running Loss: {running_loss}")

    def test(self, data):
        with torch.no_grad():
            inputs, labels = torch.tensor(data[0]).clone().detach(),data[1]
            outputs = self.model(inputs)           
            _, predicted = torch.max(outputs.data, 1)            
            labels = torch.tensor(labels)
            labels_recons = np.argmax(labels, axis=1)
            weighted_acc, report = get_classification_metrics(labels_recons, predicted)
            return predicted, weighted_acc, report
        

In [109]:
model = ANNModel() 

In [115]:
trainer = Trainer(model, [X_train_tensor,y_train_tensor], [X_val_tensor,y_val_tensor], [X_test_tensor,y_test_tensor])
trainer.train(epochs)  

C:\Users\Admin\AppData\Local\Temp\ipykernel_18640\960699775.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs, labels = torch.tensor(data[0]).clone().detach(),torch.tensor(data[1]).clone().detach()


Epoch 50, Running Loss: 2.3695267817040134e-05
Epoch 100, Running Loss: 3.3369929682812263e-06
Epoch 150, Running Loss: 1.885436559352014e-06
Epoch 200, Running Loss: 1.3342513056807823e-06
Epoch 250, Running Loss: 1.0207424496456604e-06
Epoch 300, Running Loss: 8.249803597718028e-07
Epoch 350, Running Loss: 6.868162320381231e-07
Epoch 400, Running Loss: 5.872940495864881e-07
Epoch 450, Running Loss: 5.125611551149829e-07
Epoch 500, Running Loss: 4.5058743439499864e-07
Epoch 550, Running Loss: 4.0100841150987733e-07
Epoch 600, Running Loss: 3.598140429438891e-07
Epoch 650, Running Loss: 3.2445247667206697e-07
Epoch 700, Running Loss: 2.916427793304794e-07
Epoch 750, Running Loss: 2.6393679112344364e-07
Epoch 800, Running Loss: 2.4133451726595223e-07
Epoch 850, Running Loss: 2.2201321364673554e-07
Epoch 900, Running Loss: 2.0451466543793904e-07
Epoch 950, Running Loss: 1.902970924216602e-07
Epoch 1000, Running Loss: 1.7680861864915358e-07


In [116]:
#Train
train_predicted, train_weighted_acc, train_report = trainer.test([X_train_tensor,y_train_tensor])
#Validation
val_predicted, val_weighted_acc, val_report = trainer.test([X_val_tensor,y_val_tensor])
#Test
test_predicted, test_weighted_acc, test_report = trainer.test([X_test_tensor,y_test_tensor])

C:\Users\Admin\AppData\Local\Temp\ipykernel_18640\960699775.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs, labels = torch.tensor(data[0]).clone().detach(),data[1]


In [117]:
df_result_train = pd.DataFrame(train_report, columns = ['Precision','Recall','F1 - Score', 'Accuracy - Category wise', 'Count - Category wise'])
df_result_validation = pd.DataFrame(val_report, columns = ['Precision','Recall','F1 - Score', 'Accuracy - Category wise', 'Count - Category wise'])
df_result_test = pd.DataFrame(test_report, columns = ['Precision','Recall','F1 - Score', 'Accuracy - Category wise', 'Count - Category wise'])

In [118]:
print('Training Set - Report')
print(f'Train Weighted Accuracy {train_weighted_acc}')
display(df_result_train)
print('Validation Set - Report')
print(f'Validation Weighted Accuracy {val_weighted_acc}')
display(df_result_validation)
print('Test Set - Report')
print(f'Test Weighted Accuracy {test_weighted_acc}')
display(df_result_test)

Training Set - Report
Train Weighted Accuracy 1.0


,Precision,Recall,F1 - Score,Accuracy - Category wise,Count - Category wise
0,1.0,1.0,1.0,1.0,42.0
1,1.0,1.0,1.0,1.0,162.0
2,1.0,1.0,1.0,1.0,83.0
3,1.0,1.0,1.0,1.0,40.0


Validation Set - Report
Validation Weighted Accuracy 0.4913807439239568


,Precision,Recall,F1 - Score,Accuracy - Category wise,Count - Category wise
0,0.21,0.60,0.32,0.60,5.0
1,0.73,0.68,0.70,0.68,40.0
2,0.43,0.50,0.47,0.50,20.0
3,0.12,0.06,0.08,0.06,17.0


Test Set - Report
Test Weighted Accuracy 0.4483013017460603


,Precision,Recall,F1 - Score,Accuracy - Category wise,Count - Category wise
0,0.36,0.47,0.41,0.47,17.0
1,0.66,0.45,0.53,0.45,47.0
2,0.43,0.54,0.48,0.54,28.0
3,0.07,0.09,0.08,0.09,11.0


## Conclusion  :
#### We have reverified the result of MLP classifier using manual code of ANN and achieved approximately same indication of over-fitting resulting high train accuracy and poor accuracy in both test and validation dataset.

##### (c) (Optional) Train the above-mentioned neural network model with comparatively less overfitting.(Hint: Use drop-out layers (or) early call back)

In [119]:
class RevisedANNModel(nn.Module):
    def __init__(self):
        super(RevisedANNModel, self).__init__()
        self.linear1 = nn.Linear(29, 128)        
        self.linear2 = nn.Linear(128, 128)
        self.linear3 = nn.Linear(128, 128)
        self.linear4 = nn.Linear(128, 4)
        self.dropout1 = nn.Dropout(p=0.5)
        self.dropout2 = nn.Dropout(p=0.5)
        self.dropout3 = nn.Dropout(p=0.5)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.dropout1(out)
        out = self.linear2(out)
        out = self.relu(out)
        out = self.dropout2(out)
        out = self.linear3(out)
        out = self.relu(out)
        out = self.dropout3(out)
        out = self.linear4(out)
        return out

In [120]:
revisedmodel = RevisedANNModel() 

In [125]:
trainer = Trainer(revisedmodel, [X_train_tensor,y_train_tensor], [X_val_tensor,y_val_tensor], [X_test_tensor,y_test_tensor])
trainer.train(epochs) 

C:\Users\Admin\AppData\Local\Temp\ipykernel_18640\960699775.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs, labels = torch.tensor(data[0]).clone().detach(),torch.tensor(data[1]).clone().detach()


Epoch 50, Running Loss: 0.6046668719102641
Epoch 100, Running Loss: 0.6411491813039238
Epoch 150, Running Loss: 0.8329556308211722
Epoch 200, Running Loss: 0.8802574296811191
Epoch 250, Running Loss: 0.6268070276105941
Epoch 300, Running Loss: 0.5291072440380258
Epoch 350, Running Loss: 0.6568972321934329
Epoch 400, Running Loss: 0.6922398613510918
Epoch 450, Running Loss: 0.7593597239122695
Epoch 500, Running Loss: 0.8145787373013762
Epoch 550, Running Loss: 0.722709179807339
Epoch 600, Running Loss: 0.9562974997810719
Epoch 650, Running Loss: 0.7612090692298222
Epoch 700, Running Loss: 0.8492204607416434
Epoch 750, Running Loss: 0.6477689092025172
Epoch 800, Running Loss: 0.5828931906705859
Epoch 850, Running Loss: 0.5715615677311336
Epoch 900, Running Loss: 0.6094354300446219
Epoch 950, Running Loss: 1.2282486728865647
Epoch 1000, Running Loss: 0.7765030112600261


In [160]:
#Train
train_revised_predicted, train_revised_weighted_acc, train_revised_report = trainer.test([X_train_tensor,y_train_tensor])
#Validation
val_revised_predicted, val_revised_weighted_acc, val_revised_report = trainer.test([X_val_tensor,y_val_tensor])
#Test
test_revised_predicted, test_revised_weighted_acc, test_revised_report = trainer.test([X_test_tensor,y_test_tensor])

C:\Users\Admin\AppData\Local\Temp\ipykernel_18640\960699775.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs, labels = torch.tensor(data[0]).clone().detach(),data[1]


In [162]:
df_revised_result_train = pd.DataFrame(train_revised_report, columns = ['Precision','Recall','F1 - Score', 'Accuracy - Category wise', 'Count - Category wise'])
df_revised_result_validation = pd.DataFrame(val_revised_report, columns = ['Precision','Recall','F1 - Score', 'Accuracy - Category wise', 'Count - Category wise'])
df_revised_result_test = pd.DataFrame(test_revised_report, columns = ['Precision','Recall','F1 - Score', 'Accuracy - Category wise', 'Count - Category wise'])

In [163]:
print('Revised Training Set - Report')
print(f'Revised Train Weighted Accuracy {train_revised_weighted_acc}')
display(df_revised_result_train)
print('Revised Validation Set - Report')
print(f'Revised Validation Weighted Accuracy {val_revised_weighted_acc}')
display(df_revised_result_validation)
print('Revised Test Set - Report')
print(f'Revised Test Weighted Accuracy {test_revised_weighted_acc}')
display(df_revised_result_test)

Revised Training Set - Report
Revised Train Weighted Accuracy 0.7336024831437674


,Precision,Recall,F1 - Score,Accuracy - Category wise,Count - Category wise
0,0.83,0.45,0.58,0.45,42.0
1,0.78,0.91,0.84,0.91,162.0
2,0.67,0.80,0.73,0.80,83.0
3,0.87,0.32,0.47,0.32,40.0


Revised Validation Set - Report
Revised Validation Weighted Accuracy 0.4938273459089271


,Precision,Recall,F1 - Score,Accuracy - Category wise,Count - Category wise
0,0.33,0.20,0.25,0.20,5.0
1,0.66,0.78,0.71,0.78,40.0
2,0.32,0.45,0.38,0.45,20.0
3,0.50,0.12,0.19,0.12,17.0


Revised Test Set - Report
Revised Test Weighted Accuracy 0.45253123377962584


,Precision,Recall,F1 - Score,Accuracy - Category wise,Count - Category wise
0,0.17,0.06,0.09,0.06,17.0
1,0.62,0.66,0.64,0.66,47.0
2,0.41,0.61,0.49,0.61,28.0
3,0.17,0.09,0.12,0.09,11.0


### Conclusion : We have introduced three drop out layer in our revised neural network and have noticed that train accuracy drop and test accuracy slightly increase which indicate that over-fitting has been handled by introducing drop out layer in the network. This can be further improved but considering the scope of the problem statement not proceeding any further. 